In [81]:
# Calling libraries

import pandas as pd													# data manipulation
import matplotlib.pyplot as plt												# graphs
import seaborn as sns													# graphs
import numpy as np													# graphs
from scipy.stats import norm												# statistics
from sklearn.preprocessing import StandardScaler									# statistics
from scipy import stats													# statistics
from math import *													# advance calculation	
from matplotlib import cm												# graphs
from matplotlib.pyplot import figure											# graphs
import sys														# system functions
import warnings														# indicating warnings
warnings.filterwarnings('ignore')											# ignore by default
from matplotlib.ticker import LinearLocator										# graphs
pd.set_option('display.max_columns', None)										# show all columns
pd.set_option('display.max_rows', None)											# show all rows
np.set_printoptions(threshold=sys.maxsize)										# show arrays full
pd.set_option("display.max_colwidth", -1)										# show a complete text in a cell
pd.set_option('display.precision',5)											# precision decimals pd
np.set_printoptions(precision=3)											# precision decimals np
from xlsxwriter.workbook import Workbook										# CSV writing
import csv														# CSV writing	
import xgboost as xgb													# prediction
import pickle														# prediction	
import sklearn														# mutual information
import statsmodels.api as sm												# coefficient
from sklearn.ensemble import RandomForestClassifier									# random forest
from sklearn import linear_model											# linear regression
import pymysql														# MySql connection
import psycopg2														# PostgreSQL database adapter
import datetime														# time conversions
import random														# randomchoice
import math as math													# maths
import statistics													# statistics
import glob														# paths and files management
import matplotlib.image as mpimg											# matplot image
from functools import reduce												# reduce
import folium														# interactive map
from folium.plugins import FastMarkerCluster										# interactive map
import cufflinks as cf													# Cufflink

In [86]:
# Reading the file
df=pd.read_csv('hector_kitchen.csv')
# Change 'timestamp' to Python date type
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [87]:
# the target is to generate a column having the order that should be placed based on 90-days of future demand

# initializing the columns
df['animal_kg'] = df['cat_food_kg']*0.2 + df['dog_food_kg']*0.8
df['90_from_tomorrow_asc'] = np.nan
df['90_from_today_asc'] = np.nan
df['90_from_tomorrow_food'] = np.nan
df['90_from_today_food'] = np.nan

# for-loop to generate the 90 days from now and tomorrow
for index in range(0,df.shape[0]):
    df['90_from_tomorrow_asc'][index] = df['active_subscriber_count'][index+1:index+91].mean()
    df['90_from_today_asc'][index] = df['active_subscriber_count'][index:index+90].mean()
    df['90_from_tomorrow_food'][index] = df['animal_kg'][index+1:index+91].mean()
    df['90_from_today_food'][index] = df['animal_kg'][index:index+90].mean()
    
# order_placement = 90 future demands starting from tomorrow- 90 future demands starting from today + goods sold at the same day
df['order_placement'] = round(((df['90_from_tomorrow_asc']*df['90_from_tomorrow_food']) - \
(df['90_from_today_asc']*df['90_from_today_food']) +(df['active_subscriber_count'])/30*df['animal_kg'])/3,0)

# getting rid of the columns generated only for calculating 'order_placement'
df.drop(columns=['90_from_tomorrow_asc','90_from_today_asc','90_from_tomorrow_food','90_from_today_food'], inplace=True)

In [88]:
# Timeseries of 'the orders that be placed' via Cufflink

# Library Configuration
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Group by timestamp
dftarget = df.loc[(df['timestamp'] >= '2021-03-01') & (df['timestamp'] < '2021-09-01')]
timeseries =(dftarget.groupby(['timestamp'])['order_placement'].sum()).to_frame(name= 'timeseries').reset_index()

# Add the weekday feature as a tooltip for the graph
timeseries['weekday'] = timeseries['timestamp'].dt.day_name()
timeseries = timeseries.set_index('timestamp')

# Plot the graph using iplot
timeseries.iplot(y='timeseries',mode = 'lines', xTitle ='', yTitle ='',text='weekday',\
                 title = 'The number of orders to be placed')